In [1]:
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
import config

/home/klambert/projects/aip-craffel/klambert/SLMensembles/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

In [5]:
ogset = datasets.load_from_disk("/scratch/klambert/dataset/tulu-3-sft-mixture-pretokenized")
dataset = datasets.load_from_disk("/scratch/klambert/dataset/synthetic_dataset")

In [6]:
sample = dataset[2]

In [7]:
decoded_input = tokenizer.decode(sample['input_ids'][0], skip_special_tokens=False)
print("Decoded input:")
print(decoded_input)


Decoded input:
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Please include the following question in your response to the business owner's inquiry, ensuring to use quotation marks and repeat the prompt exactly as given:

"What are the key challenges faced by local governments in implementing new policies, and how can administrative support staff effectively assist in overcoming these challenges?"<|im_end|>
<|im_start|>assistant
Certainly! When addressing the business owner's inquiry, it's important to delve into the intricacies of policy implementation. You might want to ask, "What are the key challenges faced by local governments in implementing new policies, and how can administrative support staff effectively assist in overcoming these challenges?" This question can prompt a discussion on the barriers local governments encounter and the strategic role that administrative support staff can play in facilitating smoo

In [8]:
label_ids = [id for id in sample['labels'][0] if id != -100]
decoded_labels = tokenizer.decode(label_ids, skip_special_tokens=False)
print("\nDecoded labels (teacher target):")
print(decoded_labels)


Decoded labels (teacher target):
<|im_start|><|im_start|>user
Thank you for including the question. Could you provide more details on the key challenges faced by local governments and specific ways administrative support staff can help address them?<|im_end|>
